In [1]:
import os
%cd ..
%pwd

e:\code\projects\endtoend


c:\Users\priya\anaconda3\envs\cnncancer\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'e:\\code\\projects\\endtoend'

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    valid_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentated: bool
    params_image_size: int

In [56]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'Data/train')
        valid_data = os.path.join(self.config.data_ingestion.unzip_dir, 'Data/valid')
        create_directories([config.root_dir])

        return TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            valid_data=Path(valid_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentated=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

In [58]:
import tensorflow as tf

In [64]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            shear_range=0.2,
            zoom_range=0.2,
            fill_mode='nearest'
        )

        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255
        )

        self.valid_generator = valid_datagen.flow_from_directory(
            self.config.valid_data,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
        )

        self.train_generator = train_datagen.flow_from_directory(
            self.config.training_data,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
        )
    

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps
        )

        self.save_model(self.config.trained_model_path, self.model)

In [65]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_training_config()
    training = Training(prepare_base_model_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-02-13 00:41:04,208: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-13 00:41:04,212: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-13 00:41:04,213: INFO: common: created directory at: artifacts]
[2024-02-13 00:41:04,214: INFO: common: created directory at: artifacts/training]
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
Epoch 1/15
76/76 [==============================] - 23s 286ms/step - loss: 7.5721 - accuracy: 0.5372 - val_loss: 4.5076 - val_accuracy: 0.6389
Epoch 2/15
76/76 [==============================] - 20s 257ms/step - loss: 4.4671 - accuracy: 0.6777 - val_loss: 4.2027 - val_accuracy: 0.6806
Epoch 3/15
76/76 [==============================] - 21s 283ms/step - loss: 3.5675 - accuracy: 0.7107 - val_loss: 8.5478 - val_accuracy: 0.5417
Epoch 4/15
76/76 [==============================] - 21s 279ms/step - loss: 3.3546 - accuracy: 0.7603 - val_loss: 7.8651 - val_accuracy: 0.6667
Epoch 5/15
76/76 